# Conventional volatility measure – standard deviation

In [ ]:
!pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import yfinance as yf
from scipy import stats
import statsmodels.formula.api as sm
import statsmodels.stats.stattools as tools
from scipy.stats import probplot

In [ ]:
today = datetime.today()
days = timedelta(days=1000)
start = today-days

In [ ]:
print('start:', start, '\t', 'end:', today)

start: 2019-03-29 03:10:46.870004 	 end: 2021-12-23 03:10:46.870004


In [ ]:
stk = 'EBAY'
stk = yf.download(stk, start, today)
df = stk['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [ ]:
ret = df.pct_change(1)
ret.dropna(inplace=True)
ret

Date
2019-04-01    0.014540
2019-04-02    0.006369
2019-04-03    0.001846
2019-04-04    0.001579
2019-04-05   -0.005519
                ...   
2021-12-16   -0.023929
2021-12-17   -0.000156
2021-12-20   -0.003748
2021-12-21   -0.001254
2021-12-22    0.003610
Name: Adj Close, Length: 690, dtype: float64

In [ ]:
std_annual=np.std(ret)*np.sqrt(252)

In [ ]:
print('volatility (std)=',round(std_annual,4))

volatility (std)= 0.3274


# Test of Normality

* The first value  of the result is the test statistic, and the second one is its corresponding P-value.

In [ ]:
print(stats.shapiro(ret))

# reject the null hypothesis and
# conclude that the daily returns do 
# not follow a normal distribution

(0.9656625986099243, 1.230912941019291e-11)


# Estimating fat tails

In [ ]:
print('mean =',np.mean(ret))
print('std =',np.std(ret))
print('skewness=',stats.skew(ret))
print('kurtosis=',stats.kurtosis(ret))

# the skewness is fairly symmetrical
# the kurtosis score shows outliers are present

mean = 0.0010542597966570622
std = 0.020626961241806324
skewness= -0.26987385150111703
kurtosis= 2.6374807278403765
